In [4]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import datetime

In [34]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

diff_month(datetime.date.today(),date2017)

11

In [38]:
#generate dates
date2016 = datetime.date(2016,7,3)
date2017 = datetime.date(2017,4,19)
date2012 = datetime.date(2012,9,24)
date2014 = datetime.date(2014,11,16)
thirty = (29,31)
fifteen = (14,16)
forty = (39,42)
fifty = (49,54)
sixty = (59,62)

def make_data_string(st,user_id,interval,fdate = date2017):
    datapoints = diff_month(datetime.date.today(),fdate)
    for i in range(datapoints):
        rnd = np.random.randint(*interval)
        odate = date2017 + datetime.timedelta(i*rnd)
        st += f'{user_id},{odate}|'
    return st    

In [42]:
make_data_string("",1,thirty)

'1,2017-04-19|1,2017-05-18|1,2017-06-18|1,2017-07-15|1,2017-08-17|1,2017-09-11|1,2017-10-16|1,2017-11-08|1,2017-12-07|1,2018-01-14|1,2018-02-13|'